# ***Block Deep Neural Network-Based Signal Detector for Generalized Spatial Modulation (Training)***

This code is the training of the model studied in the paper "Block Deep Neural Network-Based Signal Detector for Generalized Spatial Modulation".

***Libraries***

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import  Dense, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Accuracy
import numpy as np
import WirelessCommLib as wcl
import time

***Functions***

In [ ]:
# =====================================================================================================
# 1. Deep Learning Model
#
# ARGUMENTS
# 1-) n_x: Number of input nodes (Data Type: int)
# 2-) n_y: Number of output nodes (Data Type: int)
# 3-) n_h_list: List of the number of nodes of each hidden layer (Data Type: numpy.ndarray or list | 
# Shape: (3,))
# 4-) Np: Number of active transmit antennas (Data Type: int)
#
# OUTPUT
# - model: Deep learning model (Data Type: tensorflow.python.keras.engine.functional.Functional)
# =====================================================================================================
def Model(n_x, n_y, n_h_list, Np):
    num_hidden_layers = len(n_h_list)
    input_layer = keras.Input(shape=(n_x,), name="Input Layer")
    model_outputs = []
    model_losses = []
    model_metrics = []
    for active_antenna_index in range(Np):
        n_h1 = n_h_list[0]

        dense_layer_name = "Dense_AAI" + str(active_antenna_index + 1) + "_HLI" + str(1)
        hidden_layer = Dense(n_h1, kernel_regularizer=keras.regularizers.l2(l=0.001), activation="relu", name=dense_layer_name)(input_layer)

        bn_layer_name = "BatchNormalization_AAI" + str(active_antenna_index + 1) + "_HLI" + str(1)
        hidden_layer = BatchNormalization(name=bn_layer_name)(hidden_layer)

        for hidden_layer_index in range(1, num_hidden_layers):
            n_h = n_h_list[hidden_layer_index]

            dense_layer_name = "Dense_AAI" + str(active_antenna_index + 1) + "_HLI" + str(hidden_layer_index + 1)
            hidden_layer = Dense(n_h, kernel_regularizer=keras.regularizers.l2(l=0.001), activation="relu", name=dense_layer_name)(hidden_layer)

            bn_layer_name = "BatchNormalization_AAI" + str(active_antenna_index + 1) + "_HLI" + str(hidden_layer_index + 1)
            hidden_layer = BatchNormalization(name=bn_layer_name)(hidden_layer)

        output_layer_name = "Output_AAI" + str(active_antenna_index + 1)
        output_layer = Dense(n_y, kernel_regularizer=keras.regularizers.l2(l=0.001), activation="softmax", name=output_layer_name)(hidden_layer)

        model_outputs.append(output_layer)
        model_losses.append("categorical_crossentropy")

        metric_name = "Accuracy" + str(active_antenna_index + 1)
        model_metrics.append(Accuracy(name=metric_name))

    model = keras.Model(inputs=input_layer, outputs=model_outputs, name="B_DNN_Model")

    SGD_optimizer = SGD(lr=0.005, nesterov=True)
    model.compile(optimizer=SGD_optimizer, loss=model_losses, metrics=model_metrics)
    return model
# =====================================================================================================


# =====================================================================================================
# 2. Number Of Nodes In Each Hidden Layer
#
# ARGUMENT
# - M: Constellation size (Data Type: int | Condition: Power of 2)
#
# OUTPUT
# n_h_list: List of the number of nodes of each hidden layer (Data Type: numpy.ndarray or list | Shape:
# (3,))
# =====================================================================================================
def HiddenLayerNodes(M):
    if M == 2:
        n_h_list = [128, 64, 32]
    elif M == 4:
        n_h_list = [256, 128, 64]
    elif M == 16:
        n_h_list = [512, 256, 128]
    return n_h_list
# =====================================================================================================

***GSM Parameters***

In [ ]:
Ns = 15000000 # Number of training time slots
Nt = 2 # Number of transmit antennas

fig_name = "3a"
if fig_name == "3a":
    Np = 2 # Number of active transmit antennas
    Nr = 2 # Number of receive antennas
    M = 4 # Constellation size
    mod_type = "PSK" # Modulation type
    FVG_type = "SFVG"
elif fig_name == "3b":
    Np = 2
    Nr = 4
    M = 4
    mod_type = "PSK"
    FVG_type = "SFVG"
elif fig_name == "3c":
    Np = 2
    Nr = 64
    M = 4
    mod_type = "PSK"
    FVG_type = "SFVG"
elif fig_name == "4a-S":
    Np = 2
    Nr = 2
    M = 2
    mod_type = "PSK"
    FVG_type = "SFVG"
elif fig_name == "4a-J":
    Np = 2
    Nr = 2
    M = 2
    mod_type = "PSK"
    FVG_type = "JFVG"
elif fig_name == "4a-C":
    Np = 2
    Nr = 2
    M = 2
    mod_type = "PSK"
    FVG_type = "CFVG"
elif fig_name == "4b-2":
    Np = 2
    Nr = 2
    M = 2
    mod_type = "PSK"
    FVG_type = "SFVG"
elif fig_name == "4b-4":
    Np = 2
    Nr = 2
    M = 4
    mod_type = "PSK"
    FVG_type = "SFVG"
elif fig_name == "4b-16":
    Np = 2
    Nr = 2
    M = 16
    mod_type = "QAM"
    FVG_type = "SFVG"
else: # Enter your own parameters
    Np = 2
    Nr = 2
    M = 2
    mod_type = "PSK"
    FVG_type = "SFVG"

N_tot = wcl.Combination(Nt, Np) # Number of total transmit antenna combinations (TACs)

ns = int(np.floor(np.log2(N_tot))) # Number of spatial bits transmitted during a time-slot
m = int(np.log2(M)) # Number of information bits transmitted from a single active antenna during a time-slot
ni = Np * m # Number of total information bits transmitted during a time-slot
n_tot = ns + ni # Number of total bits transmitted during a time-slot
N = 2 ** ns # Number of illegitimate TACs

is_normalized = True
ss = wcl.Constellation(M, mod_type, is_normalized) # Signal set
TAC_set = wcl.OptimalTAC_Set(Nt, Np, N) # Optimal TAC set

***Data Preprocessing***

Input Data

In [ ]:
bit_matrix = np.random.randint(2, size=(n_tot, Ns))
if FVG_type == "SFVG":
    train_input_data = np.zeros((Ns, 2 * Nr + 2 * Nr * Np))
elif FVG_type == "JFVG":
    train_input_data = np.zeros((Ns, 1 + Np * Np))
elif FVG_type == "CFVG":
    train_input_data = np.zeros((Ns, Nr + Nr * Np))
    
for j in range(Ns):
    bit_array = bit_matrix[:, j]
    x = wcl.EncodeBits(bit_array, ss, TAC_set, ns, m, Nt, Np) # Transmitted vector
    H = wcl.Channel([Nr, Nt]) # Rayleigh fading channel
    y = np.matmul(H, x) # Received signal vector
    train_input_data[j, :] = np.concatenate((wcl.FVG(y, FVG_type), wcl.FVG(H, FVG_type)))[:, 0]

Output Data

In [ ]:
train_output_data = []
for active_antenna_index in range(Np):
    current_active_antenna_labels = np.zeros((Ns, M))
    for j in range(Ns):
        start_bit_index = ns + active_antenna_index * m
        stop_bit_index = ns + (active_antenna_index + 1) * m
        current_time_slot_bits = bit_matrix[start_bit_index : stop_bit_index, j]
        current_active_antenna_labels[j, wcl.Bin2Dec(current_time_slot_bits)] = 1
    train_output_data.append(current_active_antenna_labels)

***Deep Learning Model***

In [ ]:
if FVG_type == "SFVG":
    n_x = 2 * Nr + 2 * Nr * Np
elif FVG_type == "JFVG":
    n_x = 1 + Np * Np
elif FVG_type == "CFVG":
    n_x = Nr + Nr * Np
n_y = M
n_h_list = HiddenLayerNodes(M)
B_DNN_model = Model(n_x, n_y, n_h_list, Np)
B_DNN_model.summary()

***Training The Model***

In [ ]:
start_time = time.time()
B_DNN_model.fit(train_input_data, train_output_data, validation_split=0.25, batch_size=512, epochs=50, shuffle=True)
finish_time = time.time()
training_duration = finish_time - start_time
print("Training Time: ", training_duration, " seconds")

# Enter the path of the "Block-DNN" folder in order to save the model
B_DNN_folder_path = ""
file_path = B_DNN_folder_path + "/Trained Models"
model_name = "/B_DNN_model_Np" + str(Np) + "_Nr" + str(Nr) + "_M" + str(M) + mod_type + "_" + FVG_type + ".h5"
B_DNN_model.save(file_path + model_name)